In [ ]:
import sys
sys.path.append('./src')

In [ ]:
import tensorflow as tf
import doodle as dd

In [ ]:
tf.logging.set_verbosity('DEBUG')

In [ ]:
LABELS = [
    'apple', 'bed', 'cat', 'dog', 'eye',
    'fish', 'grass', 'hand', 'ice cream', 'jacket',
]

In [ ]:
params = {
    'train_tfrecord_file'   : 'train.tfr',
    'test_tfrecord_file'    : 'test.tfr',
    'samples_per_epoch'     : 700000,
    'save_summary_steps'    : 100,
}

In [ ]:
e = tf.estimator.Estimator(model_fn=dd.model_fn, model_dir='./model', params=params)

In [ ]:
%%time
e.train(max_steps=10, input_fn=lambda: dd.train_input_fn('./data', params))

In [ ]:
predict = e.predict(lambda: dd.eval_input_fn('./data', params))

In [ ]:
next(predict)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
predict = e.predict(lambda: dd.eval_input_fn('./data', params))
for i in range(10):
    r = next(predict)
    print(r)

In [ ]:
import tensorflow as tf
from sagemaker.tensorflow.tensorflow_serving.apis.predict_pb2 import PredictRequest
from sagemaker.tensorflow.tensorflow_serving.apis.prediction_service_pb2 import beta_create_PredictionService_stub

def create_request(value):
    request = PredictRequest()
    request.model_spec.name = 'doodle'
    request.model_spec.signature_name = 'serving_default'
    request.inputs['image'].CopyFrom(tf.make_tensor_proto(
        values=np.asarray(value).astype(np.float32).flatten(), shape=[len(value), 28, 28, 1], dtype=tf.float32))
    return request

from grpc.beta.implementations import insecure_channel

class Predictor(object):
    def __init__(self, host, port):
        self.channel = insecure_channel(host, port)
        self.stub = beta_create_PredictionService_stub(self.channel)
    def predict(self, request):
        return self.stub.Predict.future(request, 5.0)

In [ ]:
api = Predictor('127.0.0.1', 8500)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
with tf.Graph().as_default() as g, tf.Session(graph=g) as sess:
    features_tensor, labels_tensor = dd.eval_input_fn('./data', params)
    [features, labels] = sess.run([features_tensor, labels_tensor])
    for i, label in enumerate(labels):
        print('{}'.format(LABELS[label]))
        plt.imshow(np.reshape(features['image'][0], [28,28]))
        plt.show()
        break
    results = api.predict(create_request(features['image'])).result()
    predicts = np.asarray(results.outputs['classes'].int64_val)
    print(labels)
    print(predicts)